# Dev Setups -- Connecting Python and SQL

The purpose of this Jupyter notebook is to demonstrate the usefulness of connecting python to a relational database by using a python toolkit called SQLAlchemy.

***Note! The commands below were written for Python 2. Small adjustments will need to be made to some (i.e. Print statements) in Python 3.***

***First off, what is a relational database?***

Basically, it is a way to store data such that information can be retrieved from it.

MySQL and PostgreSQL are examples of relational databases.  For the purposes of an Insight project, you can use either one.

Why would you use a relational database instead of a csv or two?

**A few reasons:**

- They scale easily

-  They are easy to query

- It’s possible to do transactions in those cases where you need to write to a database, not just read from it

- Everyone in industry uses them, so you should get familiar with them, too.






***What does a relational database look like? ***

**Let's setup PostgreSQL**

We can take a look.  First we need to set up a few things. The first thing we want to do is to get a PostgreSQL server up and running.  Go to http://postgresapp.com/ and follow the three steps listed in the Quick Installation Guide. (If you aren't running a Mac, you can download PostgreSQL at http://www.postgresql.org/) 
    -- you can also use homebrew, but your path will change below --
    
**If you're on a mac, you might need to add psql to PATH**:<br>

**Edit your .bash_profile in your home directory. Since you already installed Anaconda, it should look something like:**<br>
```export PATH="/Users/YOUR_USER_NAME/anaconda/bin:$PATH"```

**Right below the line added by anaconda you can add this line:**<br>

```export PATH="/Applications/Postgres.app/Contents/Versions/latest/bin:$PATH"```

**Save and reload the bash profile**<br>
```$ source .bash_profile```

**The only user right now for PSQL is 'postgres', you can make your database and enter it with that username**<br>
```$ createdb birth_db -U postgres```<br>
```$ psql birth_db```

**If you want to make a new user for this database you can make one now. 
Note: username in the below line must match your Mac/Linux username:**<br>
```CREATE USER username SUPERUSER PASSWORD 'yourpassword'```<br>

**Exit out of PSQL (\q) and test logging in through this user:**<br>
```$ psql birth_db -h localhost -U username```<br>
```$ \c ```  (once in PSQL to check how you're logged in)<br>

We'll come back to PostgreSQL in a moment.  First, we'll set up SQLAlchemy. To get started we need to install two packages into the environment that might not be installed. Run the cell below or enter the commands (without !) into the command line. 

Note that if you did an Anaconda installation, sqlalchemy_utils is only available through pip, and if you didn't install pip into your environment (dev_setups_conda-part1.html) you will run into problems. Also, you need to install psycopg2 using conda, otherwise you will probably run into different problems. If you mainly installed packages using pip, change the next commands to reflect that.

In jupyter you can run code in the command line with the "!" special character as you'll see in the next cell.  We do this here for ease but it's generally considered poor practice.

In [0]:
!pip install sqlalchemy_utils 
!conda install psycopg2 -y

In [0]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

## (Optional) If Postgres isn't launched on startup 

**To have launchd start postgresql at login: **<br>
```ln -sfv /usr/local/opt/postgresql/*.plist ~/Library/LaunchAgents``` <br><br>
**Then to load postgresql now: **<br>
```launchctl load ~/Library/LaunchAgents/homebrew.mxcl.postgresql.plist``` <br><br>
**Or, if you don't want/need launchctl, you can just run: **<br>
```postgres -D /usr/local/var/postgres``` <br>
**into the command line and also look at [this page](http://postgresguide.com/) for more details.**


## Interfacing with PSQL through python

Update your username and password in the cell below. Then run each cell. 

In [0]:
#In Python: Define your username and password used above. I've defined the database name (we're 
#using a dataset on births, so I call it birth_db). 
dbname = 'birth_db'
username = 'patrickphelps'
pswd = 'R3combination'

In [0]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print(engine.url)
# Replace localhost with IP address if accessing a remote server

In [0]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
print(engine.url)


<h1> Getting some data </h1>
Time to get some data, head over to https://drive.google.com/open?id=1YlN9vG2qY1DdtC9ni4ItPhoYm7GHTNfu and download the births2012_downsampled.csv.

In [0]:
# load a database from the included CSV
# edit the string below to account for where you saved the csv.
csv_path = '../Downloads/births2012_downsampled.csv'
birth_data = pd.read_csv(csv_path)

In [0]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
## df is any pandas dataframe 
birth_data.to_sql('birth_data_table', engine, if_exists='replace')

The above line (to_sql) is doing a lot of heavy lifting.  It's reading a dataframe, it's creating a table, and adding the data to the table.  So ** SQLAlchemy is quite useful! **

### How this works outside of python:

** open up the PostgreSQL app, click on the "Open psql" button in the bottom right corner, ** <br>
or alternatively type <br>
```$ psql birth_db -h localhost -U username``` <br> 
into the command line  

**Type the following into the terminal that opens up**

`$ \c birth_db`

**You should see something like the following**

`$ You are now connected to database "birth_db" as user "username".`



**Then try the following query:**

`$ SELECT * FROM birth_data_table;`

### You can see the table we created!  But it's kinda ugly and hard to read (type 'q' in terminal to end long output).

**You can try a few other sample queries.  Before you type in each one, ask yourself what you think the output will look like:**

`SELECT * FROM birth_data_table WHERE infant_sex='M';`

`SELECT COUNT(infant_sex) FROM birth_data_table WHERE infant_sex='M';`

`SELECT COUNT(gestation_weeks), infant_sex FROM birth_data_table WHERE infant_sex = 'M' GROUP BY gestation_weeks, infant_sex;`

`SELECT gestation_weeks, COUNT(gestation_weeks) FROM birth_data_table WHERE infant_sex = 'M' GROUP BY gestation_weeks;`

In [0]:
## Now try the same queries, but in python!

# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)
`
# query:
sql_query = """
SELECT * FROM birth_data_table WHERE delivery_method='Cesarean';
"""
birth_data_from_sql = pd.read_sql_query(sql_query,con)

birth_data_from_sql.head()

### Is one method of querying the data faster than the other?  Probably not for the amount of data you can fit on your machine.

In [0]:
import time

t0 = time.time()
birth_data_from_sql = pd.read_sql_query(sql_query,con)
t1 = time.time()
total = t1-t0
print('total time take: ' + str(total) + ' seconds')

In [0]:
birth_data = pd.read_csv(csv_path)

t0 = time.time()
birth_data=birth_data.loc[(birth_data['delivery_method'] == 'Cesarean')]
t1 = time.time()
total = t1-t0
print('total time take: ' + str(total) + ' seconds')

**This should have given you a quick taste of how to use SQLALchemy, as well as how to run a few SQL queries both at the command line and in python.  You can see that pandas is actually a little faster than PostgreSQL here - that is because of the extra time it takes to communicate between python and PostGreSQL.  But as your database gets bigger (and certainly when it's too large to store in memory), working with relational databases becomes a necessity.**
